In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 27.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.1 MB/s 
     |████████████████████████████████| 120 kB 71.3 MB/s 


In [ ]:
import ml_metrics as metrics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_name = '/content/drive/My Drive/quote_recommendation'

In [4]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [5]:
# Import generic wrappers
from transformers import AutoTokenizer 
# Define the model repo
model_name = "SpanBERT/spanbert-base-cased" 
tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)

In [6]:
import json
with open("/content/drive/My Drive/quote_recommendation/gandhi.json", "r") as read_it:
     data = json.load(read_it)

list_mp_val = []
for key,val in data.items():
    temp = {
    "answers":
        {
            "text":"",
            "answer_start":0,
            "answer_end":0
        },
    "context":"",
    "questions":""
    }
    temp["answers"]["text"] = val["quote"]
    temp["answers"]["answer_start"] = val["start_index"]
    temp["answers"]["answer_end"] = val["end_index"]
    temp["questions"] = key
    temp["context"] = val["paragraph"]
    list_mp_val.append(temp)

  
def read_quotus(list_mp):
    contexts = []
    questions = []
    answers = []
    for val in list_mp:
        contexts.append(val["context"])
        questions.append(val["questions"])
        answers.append(val["answers"])
    return contexts, questions, answers


val_contexts, val_questions, val_answers = read_quotus(list_mp_val)

In [7]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:
answers_starts = []
answers_ends = []
actual_starts , actual_ends = [],[]
#n = len(val_questions)
for id in range(30):
    question = val_questions[id]
    context = val_contexts[id]

    actual_starts.append([val_answers[id]["answer_start"]])
    actual_ends.append([val_answers[id]["answer_end"]])

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt',max_length=512)
    #inputs.to(device)
    outputs = model(**inputs)

    # print(outputs.__dict__)
    print(f"================================================================================================================")
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    #print(f"answer start {answer_start}")
    answers_starts.append(torch.topk(outputs[0], 5)[1][0])
    #print(f"predicted answer starts {answers_starts}")
    #print(f"actual answer starts {actual_starts}")
    answer_end = torch.argmax(outputs[1])
    #print(f"answer end {answer_end}")
    answers_ends.append(torch.topk(outputs[1], 5)[1][0])
    #print(f"answer ends {answers_ends}")
    #print(f"actual answer ends {actual_ends}")
    print("Actual result ------------------------------")
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][actual_starts[0][0]:actual_ends[0][0]+1]))
    print(predicted_answer)
    #print(context[answer_start:answer_end+1])
    print(f"============================================================")
    print("Top 5 predicted results")
    for i in range(5):
      print(f"pridicted answer {i+1} -> {tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answers_starts[id][i]:answers_ends[id][i]+1]))}")
      #print(f"pridicted answer {i+1} -> {context[answers_starts[id][i]:answers_ends[id][i]+1]}")


Actual result ------------------------------
getting what one wants, and the other is getting it. the last is much the worst ; the last is a real tragedy! but i am interested to hear she does not love you. how long could you love a woman who didn't love you, cecil? [SEP]
Top 5 predicted results
pridicted answer 1 -> i wish you good luck,
pridicted answer 2 -> 
pridicted answer 3 -> 
pridicted answer 4 -> 
pridicted answer 5 -> 
Actual result ------------------------------
nobody will waste his life in accumulating things, and the symbols for things. one will live. to live is the rarest thing in the world. most people exist, that is all. [SEP]
Top 5 predicted results
pridicted answer 1 -> most people exist,
pridicted answer 2 -> nobody will waste their life accumulating and symbols for things. [SEP] with the abolition of private property, then, we shall have true, beautiful, healthy individualism. nobody will waste his life in accumulating things, and the symbols for things. one will li